# GrassClover-Style Generation with Stable Diffusion
Este notebook implementa geração de imagens sintéticas de pastagens brasileiras usando **Stable Diffusion**, seguindo o estilo visual do **GrassClover Dataset** (Skovsen et al., CVPR 2019).
## Objetivos:
- Gerar imagens **top-down** de pastagens com Stable Diffusion
- Adaptar para **gramíneas brasileiras** (Brachiaria, Panicum, Cynodon)
- Seguir **metodologia GrassClover** (densidade, perspectiva, resolução)
- **Ultra-compatível** com Google Colab (debugging extensivo)
## Referência:
- **Paper**: Skovsen et al. "The GrassClover Image Dataset for Semantic and Hierarchical Species Understanding in Agriculture" (CVPR Workshops, 2019)
- **Adaptação**: Espécies temperadas → Tropicais brasileiras


## Setup Ultra-Compatível para Colab
**IMPORTANTE**: Este notebook foi desenvolvido para máxima compatibilidade com Google Colab Free/Pro.

In [ ]:
import sys
import platform
from datetime import datetime
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False


In [ ]:
device = None
dev_type = "unknown"
hw_info = {}

# Tentar importar torch primeiro
try:
    import torch
    torch_ok = True
    hw_info['pytorch_version'] = torch.__version__
except ImportError:
    torch_ok = False

if torch_ok:
    # Verificar TPU primeiro
    try:
        import torch_xla
        import torch_xla.core.xla_model as xm
        if xm.xrt_world_size() > 1:
            device = xm.xla_device()
            dev_type = "tpu"
            hw_info.update({
                'dev_type': 'tpu',
                'tpu_cores': xm.xrt_world_size(),
                'device': str(device)
            })
        else:
            # TPU disponível mas não configurado
            device = torch.device("cpu")
            dev_type = "cpu"
    except ImportError:
        # TPU não disponível, tentar GPU
        pass
    except Exception as e:
        print(f"⚠️  Erro ao verificar TPU: {e}")
    
    # Se não conseguiu TPU, tentar GPU
    if device is None:
        cuda_available = torch.cuda.is_available()
        if cuda_available:
            gpu_count = torch.cuda.device_count()
            for i in range(gpu_count):
                gpu_name = torch.cuda.get_device_name(i)
                gpu_memory = torch.cuda.get_device_properties(i).total_memory
                gpu_memory_gb = gpu_memory / (1024**3)
            
            device = torch.device("cuda")
            dev_type = "gpu"
            hw_info.update({
                'dev_type': 'gpu',
                'gpu_count': gpu_count,
                'gpu_name': torch.cuda.get_device_name(0),
                'gpu_memory_gb': torch.cuda.get_device_properties(0).total_memory / (1024**3),
                'device': str(device)
            })
            torch.cuda.empty_cache()
        else:
            # Fallback para CPU
            device = torch.device("cpu")
            dev_type = "cpu"
    
    # Se ainda não definiu device, usar CPU
    if device is None:
        device = torch.device("cpu")
        dev_type = "cpu"
        hw_info.update({
            'dev_type': 'cpu',
            'device': str(device)
        })

    # Print do status do hardware
    if dev_type == "cpu" and in_colab:
        print("⚠️ CPU detectado - Considere ativar GPU no Colab")
    elif dev_type == "tpu":
        print("🔥 TPU detectado")
    elif dev_type == "gpu":
        print(f"🚀 GPU detectado: {hw_info.get('gpu_name', 'Unknown')}")
        
else:
    device = None
    dev_type = "none"

print(f"🔧 Hardware configurado: {dev_type.upper()}")
print(f"📱 Device: {device}")
if hw_info:
    print(f"ℹ️  Info: {hw_info}")

In [ ]:
# 🔐 AUTENTICAÇÃO HUGGINGFACE OBRIGATÓRIA
print("🔐 AUTENTICAÇÃO HUGGINGFACE OBRIGATÓRIA")
print("=" * 60)
print("⚠️  IMPORTANTE: Este notebook REQUER login no HuggingFace para funcionar!")
print("📋 Você DEVE aceitar os termos em: https://huggingface.co/stabilityai/stable-diffusion-3.5-large")
print("🔑 Você DEVE ter um token HuggingFace válido")
print("=" * 60)

try:
    from huggingface_hub import notebook_login, whoami
    
    print("⏳ Iniciando processo de login...")
    
    # Forçar login interativo - isso vai parar e pedir o token
    notebook_login()
    
    # Verificar se o login foi bem-sucedido
    try:
        user_info = whoami()
        print(f"✅ Login realizado com sucesso!")
        print(f"👤 Usuário logado: {user_info.get('name', 'N/A')}")
        HUGGINGFACE_LOGGED_IN = True
    except Exception as e:
        print(f"❌ Falha na verificação do login: {e}")
        print("🚫 PARANDO EXECUÇÃO - LOGIN NECESSÁRIO")
        raise SystemExit("Login no HuggingFace é obrigatório para continuar")
    
except ImportError as e:
    print(f"❌ Erro ao importar huggingface_hub: {e}")
    print("📦 Instalando huggingface_hub...")
    import subprocess
    result = subprocess.run(["pip", "install", "huggingface_hub", "--upgrade", "--quiet"], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        from huggingface_hub import notebook_login, whoami
        print("✅ huggingface_hub instalado")
        
        # Tentar login novamente após instalação
        notebook_login()
        try:
            user_info = whoami()
            print(f"✅ Login realizado com sucesso após instalação!")
            print(f"👤 Usuário logado: {user_info.get('name', 'N/A')}")
            HUGGINGFACE_LOGGED_IN = True
        except:
            print("🚫 PARANDO EXECUÇÃO - LOGIN NECESSÁRIO")
            raise SystemExit("Login no HuggingFace é obrigatório para continuar")
    else:
        print("❌ Falha ao instalar huggingface_hub")
        raise SystemExit("Não foi possível instalar huggingface_hub")

except Exception as e:
    print(f"❌ Erro durante o login: {e}")
    print("🚫 PARANDO EXECUÇÃO - LOGIN NECESSÁRIO")
    print("\n💡 SOLUÇÕES:")
    print("1. Certifique-se de que tem um token HuggingFace válido")
    print("2. Acesse: https://huggingface.co/settings/tokens")
    print("3. Aceite os termos do modelo: https://huggingface.co/stabilityai/stable-diffusion-3.5-large")
    print("4. Execute esta célula novamente")
    raise SystemExit("Login no HuggingFace é obrigatório para continuar")

print("=" * 60)
print("🎉 AUTENTICAÇÃO CONCLUÍDA - NOTEBOOK PODE CONTINUAR")
print("=" * 60)

In [ ]:
# 🚀 STABLE DIFFUSION 3.5 LARGE - CONFIGURAÇÃO EXCLUSIVA
print("🚀 Carregando Stable Diffusion 3.5 Large...")
print("⚠️  APENAS SD3.5 SERÁ USADO - SEM FALLBACK PARA VERSÕES ANTIGAS")

def load_stable_diffusion_35_only():
    """
    Carrega APENAS SD3.5 Large - sem fallback para outras versões
    """
    # Verificar se o login foi feito
    if not HUGGINGFACE_LOGGED_IN:
        print("🚫 ERRO: Login no HuggingFace é obrigatório!")
        print("📋 Execute a célula anterior para fazer login")
        raise SystemExit("Login necessário para continuar")
    
    try:
        # Imports específicos para SD3.5
        from diffusers import StableDiffusion3Pipeline
        import torch
        
        print("⏳ Carregando SD3.5 Large (pode demorar alguns minutos)...")
        
        # Determinar device_map baseado no hardware
        if torch.cuda.is_available():
            # Para GPU, usar device específico ao invés de "auto"
            device_map_strategy = None  # Usar .to(device) depois
            torch_dtype = torch.bfloat16
            print("🚀 Usando GPU com bfloat16")
        else:
            device_map_strategy = None
            torch_dtype = torch.float32
            print("💻 Usando CPU com float32")
        
        # Configurações otimizadas para SD3.5
        load_kwargs = {
            "torch_dtype": torch_dtype,
            "low_cpu_mem_usage": True,
        }
        
        # Adicionar device_map apenas se necessário
        if device_map_strategy:
            load_kwargs["device_map"] = device_map_strategy
        
        print("📦 Fazendo download/carregamento do modelo...")
        pipe = StableDiffusion3Pipeline.from_pretrained(
            "stabilityai/stable-diffusion-3.5-large", 
            **load_kwargs
        )
        
        # Mover para device manualmente se não usou device_map
        if not device_map_strategy:
            print(f"📱 Movendo modelo para {device}...")
            pipe = pipe.to(device)
            
        print("✅ SD3.5 Large carregado com sucesso!")
        return pipe, "3.5"
        
    except Exception as e:
        print(f"❌ ERRO ao carregar SD3.5: {e}")
        print(f"🔍 Tipo do erro: {type(e).__name__}")
        
        # Análise específica de erros
        error_str = str(e).lower()
        
        if "auto not supported" in error_str:
            print("💡 DIAGNÓSTICO: Problema com device_map='auto'")
            print("🔧 CORREÇÃO: Removendo device_map e usando .to(device)")
            
        elif "requires you to be logged in" in error_str or "authentication" in error_str:
            print("💡 DIAGNÓSTICO: Problema de autenticação")
            print("🔧 CORREÇÃO: Verifique se fez login correto na célula anterior")
            
        elif "out of memory" in error_str or "oom" in error_str:
            print("💡 DIAGNÓSTICO: Problema de memória")
            print("🔧 CORREÇÃO: SD3.5 Large requer ~12GB+ VRAM")
            
        elif "connection" in error_str or "network" in error_str:
            print("💡 DIAGNÓSTICO: Problema de conexão")
            print("🔧 CORREÇÃO: Verifique sua conexão com internet")
            
        # NÃO FAZER FALLBACK - PARAR AQUI
        print("\n🚫 PARANDO EXECUÇÃO - SD3.5 É OBRIGATÓRIO")
        print("💭 Este notebook foi projetado especificamente para SD3.5")
        print("🔄 SOLUÇÕES:")
        print("  1. Verifique seu login HuggingFace")
        print("  2. Aceite os termos do modelo")
        print("  3. Certifique-se de ter GPU com VRAM suficiente")
        print("  4. Tente reiniciar o runtime se necessário")
        
        raise SystemExit(f"SD3.5 é obrigatório. Erro: {e}")

# Definir TORCH_DTYPE se não existir
if 'TORCH_DTYPE' not in locals():
    if device.type == "cuda":
        TORCH_DTYPE = torch.bfloat16
        print(f"🔢 TORCH_DTYPE configurado: {TORCH_DTYPE} (GPU)")
    else:
        TORCH_DTYPE = torch.float32
        print(f"🔢 TORCH_DTYPE configurado: {TORCH_DTYPE} (CPU)")

# Carregar pipeline - APENAS SD3.5
pipe, sd_version = load_stable_diffusion_35_only()

if pipe is not None:
    print(f"\n🎉 Pipeline SD{sd_version} carregado com sucesso!")
    
    # Configurar scheduler específico para SD3.5
    print("🔧 Configurando scheduler otimizado...")
    # SD3.5 já vem com scheduler adequado, não precisamos trocar
    print("✅ Scheduler SD3.5 mantido (nativo do modelo)")

    # Configurações específicas para SD3.5
    GENERATION_PARAMS_CURRENT = {
        'width': 1024,           # SD3.5 funciona melhor em resolução maior
        'height': 1024,
        'num_inference_steps': 28,  # Padrão SD3.5
        'guidance_scale': 4.5,      # Ajustado para SD3.5
        'num_images_per_prompt': 1,
        'eta': 0.0,
        'generator_seed': 42
    }
    
    print("⚙️ Parâmetros SD3.5 configurados:")
    for param, value in GENERATION_PARAMS_CURRENT.items():
        print(f"  • {param}: {value}")
    
    # Otimizações de memória específicas
    try:
        if hasattr(pipe, 'enable_attention_slicing'):
            pipe.enable_attention_slicing()
            print("⚡ Attention slicing ativado")
            
        if hasattr(pipe, 'enable_memory_efficient_attention') and device.type == "cuda":
            pipe.enable_memory_efficient_attention()  
            print("💾 Memory efficient attention ativado")
            
    except Exception as e:
        print(f"⚠️ Aviso nas otimizações: {e}")
    
    print(f"\n🎉 SD3.5 PRONTO PARA GERAÇÃO!")
    PIPELINE_READY = True
    
else:
    print("🚫 SISTEMA PARADO - SD3.5 NÃO FOI CARREGADO")
    PIPELINE_READY = False

In [ ]:
essential_packages = [
    "torch",
    "torchvision", 
    "diffusers",
    "transformers",
    "accelerate",
    "pillow",
    "matplotlib",
    "numpy"
]

def install_package(package_name, quiet=True):
    """Instala pacote com debugging"""
    import subprocess
    try:
        cmd = ["pip", "install", package_name]
        if quiet:
            cmd.append("--quiet")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            return True
        else:
            print(f"❌ Erro ao instalar {package_name}:")
            print(result.stderr[:200])
            return False
    except subprocess.TimeoutExpired:
        print(f"⏱️ Timeout ao instalar {package_name}")
        return False
    except Exception as e:
        print(f"💥 Erro inesperado ao instalar {package_name}: {e}")
        return False

print("📦 Verificando e instalando pacotes essenciais...")

for package in essential_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} já instalado")
    except ImportError:
        print(f"📦 Instalando {package}...")
        success = install_package(package)
        if not success:
            print(f"❌ Falhou ao instalar {package} - continuando mesmo assim")
            break

print("✅ Verificação de pacotes concluída!")

In [ ]:
# 🔄 REIMPORTAÇÃO E VERIFICAÇÃO FINAL COM CONFIGURAÇÕES OTIMIZADAS
print("🔄 Verificação final do sistema...")

# Imports essenciais com debugging
try:
    import torch
    import torchvision.transforms as transforms
    print("✅ PyTorch importado")
    
    # Reconfigurar device após instalação se necessário
    if not 'device' in locals() or device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            dev_type = "gpu"
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            print(f"🚀 GPU detectada: {gpu_name} ({gpu_memory:.1f}GB)")
            
            # Configuração otimizada para GPU
            torch.backends.cudnn.benchmark = True
            torch.cuda.empty_cache()
        else:
            device = torch.device("cpu")
            dev_type = "cpu"
            print("💻 Usando CPU")
    
    # Configurar dtype baseado no hardware
    if dev_type == "gpu":
        TORCH_DTYPE = torch.bfloat16  # GPU: usar half precision
    elif dev_type == "tpu":
        TORCH_DTYPE = torch.float32  # TPU: full precision recomendado
    else:
        TORCH_DTYPE = torch.float32  # CPU: full precision
        
    print(f"🔢 Dtype configurado: {TORCH_DTYPE}")
    
except ImportError as e:
    print(f"❌ Erro PyTorch: {e}")
    device = None
    dev_type = "none"
    TORCH_DTYPE = None

try:
    from diffusers import StableDiffusion3Pipeline, DPMSolverMultistepScheduler
    print("✅ Diffusers importado")
    diffusers_ok = True
except ImportError as e:
    print(f"❌ Erro Diffusers: {e}")
    diffusers_ok = False

try:
    from PIL import Image, ImageEnhance, ImageFilter
    import matplotlib.pyplot as plt
    import numpy as np
    print("✅ Bibliotecas de imagem importadas")
    libs_ok = True
except ImportError as e:
    print(f"❌ Erro bibliotecas básicas: {e}")
    libs_ok = False

# Configuração de ambiente Hugging Face
try:
    import os
    os.environ["DISABLE_TELEMETRY"] = "1"
    os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
    
    # Configurar cache offline se necessário
    if in_colab:
        os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Permitir downloads no Colab
        os.environ["HF_HUB_OFFLINE"] = "0"
    print("✅ Configuração HF aplicada")
except Exception as e:
    print(f"⚠️ Aviso na configuração HF: {e}")

ALL_READY = device is not None and diffusers_ok and libs_ok

if ALL_READY:
    print("✅ Sistema totalmente configurado e pronto!")
    print(f"📱 Device: {device}")
    print(f"🔧 Tipo: {dev_type}")
else:
    print("❌ Sistema não está completamente configurado")
    if device is None:
        print("  - Device não configurado")
    if not diffusers_ok:
        print("  - Diffusers não disponível")
    if not libs_ok:
        print("  - Bibliotecas básicas com problema")
    
    if dev_type == "cpu" and in_colab:
        print("⚠️ Executando em CPU - Performance limitada")
        print("💡 Considere ativar GPU no Runtime do Colab")

print("=" * 60)

## Pipeline Stable Diffusion para GrassClover
Configuração do pipeline otimizado para geração de pastagens no estilo GrassClover.

# CONFIGURAÇÃO AVANÇADA DO PIPELINE STABLE DIFFUSION
print("🎨 Configurando Stable Diffusion Pipeline...\n")
# Parâmetros do pipeline baseados no hardware detectado
MODEL_ID = "runwayml/stable-diffusion-v1-5"  # Modelo base confiável
LOW_CPU_MEM_USAGE = True
ENABLE_ATTENTION_SLICING = True
# Configurações específicas por hardware
if device_type == "gpu":
    ENABLE_MODEL_CPU_OFFLOAD = False
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = True
elif device_type == "tpu":
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = False  # TPU pode ter problemas com compile
else:  # CPU
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = True
    USE_TORCH_COMPILE = False
print(f"📦 Modelo: {MODEL_ID}")
print(f"🔢 Dtype: {TORCH_DTYPE}")
print(f"💾 Low CPU mem: {LOW_CPU_MEM_USAGE}")
print(f"⚡ Attention slicing: {ENABLE_ATTENTION_SLICING}")
print(f"🏃 Model CPU offload: {ENABLE_MODEL_CPU_OFFLOAD}")
print(f"🔥 Hardware: {device_type.upper()}")
# Função para carregar pipeline com debugging e bypass de autenticação
def load_stable_diffusion_pipeline():
    """Carrega pipeline com máximo debugging e configurações otimizadas"""
    try:
        print("⏳ Carregando modelo...")
        # Configurações para bypass de problemas de autenticação
        load_kwargs = {
            "torch_dtype": TORCH_DTYPE,
            "low_cpu_mem_usage": LOW_CPU_MEM_USAGE,
            "use_safetensors": True,
        }
        # Configurações específicas para resolver problemas HF
        try:
            # Tentar com autenticação padrão primeiro
            pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com autenticação padrão")
        except Exception as auth_error:
            print(f"⚠️  Problema de autenticação: {str(auth_error)[:100]}...")
            print("🔄 Tentando download forçado...")
            # Bypass de problemas de autenticação
            load_kwargs["use_auth_token"] = False
            load_kwargs["force_download"] = False
            load_kwargs["resume_download"] = True
            pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com bypass de autenticação")
        # Mover para device
        pipe = pipe.to(device)
        print(f"🚀 Pipeline movido para {device}")
        # Otimizações baseadas no hardware
        if ENABLE_ATTENTION_SLICING:
            pipe.enable_attention_slicing()
            print("⚡ Attention slicing habilitado")
        if ENABLE_MODEL_CPU_OFFLOAD and device_type != "cpu":
            try:
                pipe.enable_model_cpu_offload()
                print("💾 Model CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  CPU offload falhou: {e}")
        if ENABLE_SEQUENTIAL_CPU_OFFLOAD and device_type == "cpu":
            try:
                pipe.enable_sequential_cpu_offload()
                print("🔄 Sequential CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  Sequential offload falhou: {e}")
        # Scheduler otimizado
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        print("🔧 Scheduler otimizado (DPMSolver)")
        # Configurações de memória específicas
        if device_type == "gpu":
            torch.cuda.empty_cache()
            allocated = torch.cuda.memory_allocated() / (1024**3)
            cached = torch.cuda.memory_reserved() / (1024**3)
            print(f"💾 GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB")
            # Verificar se há memória suficiente
            if allocated > 10.0:  # >10GB pode causar problemas
                print(f"⚠️  Alta utilização de memória GPU!")
        elif device_type == "tpu":
            print("🔥 TPU configurado - memória gerenciada automaticamente")
        else:  # CPU
            print("💻 CPU mode - sem monitoramento de GPU memory")
        print("\n✅ Pipeline configurado com sucesso!")
        return pipe
    except Exception as e:
        print(f"💥 ERRO ao carregar pipeline: {e}")
        print(f"\n📋 DEBUG COPY/PASTE:")
        print(f"Model: {MODEL_ID}")
        print(f"Device: {device} ({device_type})")
        print(f"Dtype: {TORCH_DTYPE}")
        print(f"Hardware info: {hardware_info if 'hardware_info' in locals() else 'N/A'}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        # Sugestões baseadas no tipo de erro
        error_str = str(e).lower()
        if "authentication" in error_str or "token" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE TOKEN:")
            print(f"1. Ignore o warning - o modelo deve funcionar mesmo assim")
            print(f"2. Ou configure HF_TOKEN manualmente se necessário")
        elif "memory" in error_str or "cuda" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE MEMÓRIA:")
            print(f"1. Reduzir batch_size")
            print(f"2. Usar torch.float16 se estiver usando float32")
            print(f"3. Fechar outros notebooks no Colab")
        elif "module" in error_str or "import" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE MÓDULO:")
            print(f"1. Re-executar células de instalação")
            print(f"2. Restart runtime se necessário")
        return None
# Carregar pipeline
if ALL_READY:
    pipe = load_stable_diffusion_pipeline()
    PIPELINE_READY = pipe is not None
else:
    pipe = None
    PIPELINE_READY = False
    print("❌ Sistema não está pronto para carregar pipeline")
print("=" * 60)

In [ ]:
# 🎨 STATUS DO SISTEMA
print("🎨 Verificando status do sistema SD3.5...")

# Verificar se todos os pré-requisitos estão atendidos
def check_system_status():
    """Verifica se o sistema está pronto para usar SD3.5"""
    
    status = {
        'huggingface_login': False,
        'pipeline_loaded': False,
        'device_ready': False,
        'all_ready': False
    }
    
    # Verificar login HuggingFace
    if 'HUGGINGFACE_LOGGED_IN' in globals() and HUGGINGFACE_LOGGED_IN:
        print("✅ HuggingFace: Login OK")
        status['huggingface_login'] = True
    else:
        print("❌ HuggingFace: Login necessário")
        return status
    
    # Verificar pipeline
    if 'PIPELINE_READY' in globals() and PIPELINE_READY:
        print("✅ Pipeline SD3.5: Carregado")
        status['pipeline_loaded'] = True
    else:
        print("❌ Pipeline SD3.5: Não carregado")
        return status
    
    # Verificar device
    if 'device' in globals() and device is not None:
        print(f"✅ Device: {device} ({dev_type})")
        status['device_ready'] = True
    else:
        print("❌ Device: Não configurado")
        return status
    
    # Verificar se tudo está pronto
    if all([status['huggingface_login'], status['pipeline_loaded'], status['device_ready']]):
        status['all_ready'] = True
        print("🎉 Sistema completamente pronto para geração!")
    else:
        print("⚠️ Sistema não está completamente configurado")
    
    return status

# Executar verificação
system_status = check_system_status()

if system_status['all_ready']:
    print(f"\n📋 CONFIGURAÇÃO FINAL:")
    print(f"  • Modelo: Stable Diffusion 3.5 Large")
    print(f"  • Device: {device}")
    print(f"  • Dtype: {TORCH_DTYPE}")
    print(f"  • Resolução: {GENERATION_PARAMS_CURRENT['width']}x{GENERATION_PARAMS_CURRENT['height']}")
    print(f"  • Steps: {GENERATION_PARAMS_CURRENT['num_inference_steps']}")
    print(f"  • Guidance: {GENERATION_PARAMS_CURRENT['guidance_scale']}")
    
    print(f"\n🚀 PRONTO PARA GERAR IMAGENS GRASSCLOVER!")
    
else:
    print(f"\n🚫 SISTEMA NÃO ESTÁ PRONTO")
    print(f"📋 Execute as células anteriores na ordem:")
    print(f"  1. Login HuggingFace (obrigatório)")
    print(f"  2. Configuração de hardware")
    print(f"  3. Carregamento do SD3.5")

print("=" * 60)

In [ ]:
# 📚 DOWNLOAD DO DATASET GRASSCLOVER ORIGINAL
print("📚 Baixando dataset GrassClover original do Kaggle...\n")

# Inicializar variável globalmente
GRASSCLOVER_DATASET_PATH = None

try:
    # Instalar kagglehub se necessário
    try:
        import kagglehub
        print("✅ kagglehub já disponível")
    except ImportError:
        print("📦 Instalando kagglehub...")
        import subprocess
        result = subprocess.run(["pip", "install", "kagglehub", "--quiet"], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            import kagglehub
            print("✅ kagglehub instalado com sucesso")
        else:
            print(f"❌ Erro ao instalar kagglehub: {result.stderr}")
            raise ImportError("kagglehub installation failed")
    
    # Download do dataset
    print("⏳ Fazendo download do GrassClover dataset...")
    print("💡 Isso pode demorar alguns minutos na primeira vez...")
    
    dataset_path = kagglehub.dataset_download("usharengaraju/grassclover-dataset")
    
    if dataset_path and os.path.exists(dataset_path):
        GRASSCLOVER_DATASET_PATH = dataset_path
        print(f"✅ Dataset baixado com sucesso!")
        print(f"📁 Localização: {dataset_path}")
        
        # Explorar estrutura do dataset
        print(f"\n📋 Estrutura do dataset:")
        
        total_files = 0
        for root, dirs, files in os.walk(dataset_path):
            level = root.replace(dataset_path, '').count(os.sep)
            indent = '  ' * level
            folder_name = os.path.basename(root) if root != dataset_path else 'grassclover-dataset'
            
            # Contar apenas arquivos de imagem
            image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if image_files:
                print(f"{indent}{folder_name}/ ({len(image_files)} imagens)")
                total_files += len(image_files)
                
                # Mostrar alguns exemplos de nomes
                for file in image_files[:3]:
                    print(f"{indent}  • {file}")
                if len(image_files) > 3:
                    print(f"{indent}  • ... e mais {len(image_files)-3} imagens")
            elif dirs:
                print(f"{indent}{folder_name}/")
        
        print(f"\n📊 Total: {total_files} imagens encontradas")
        
        if total_files == 0:
            print("⚠️  Nenhuma imagem encontrada no dataset baixado")
            GRASSCLOVER_DATASET_PATH = None
    else:
        print("❌ Download retornou path inválido")
        GRASSCLOVER_DATASET_PATH = None
    
except Exception as e:
    print(f"❌ Erro ao baixar dataset: {e}")
    print(f"\n📋 DEBUG INFO:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)[:200]}...")
    
    # Manter variável definida como None
    GRASSCLOVER_DATASET_PATH = None
    
    print(f"\n💡 SOLUÇÕES POSSÍVEIS:")
    print(f"1. Verificar conectividade com internet")
    print(f"2. Tentar novamente em alguns minutos")
    print(f"3. Verificar se Kaggle está acessível")
    print(f"4. OPCIONAL: Upload manual de imagens GrassClover")

# Status final
if GRASSCLOVER_DATASET_PATH:
    print(f"\n✅ Dataset GrassClover disponível para análise!")
else:
    print(f"\n⚠️  Continuando sem dataset GrassClover")
    print(f"🎯 O notebook ainda funcionará, mas sem calibração visual")

print("=" * 60)

In [ ]:
# 🔍 ANÁLISE VISUAL DO DATASET GRASSCLOVER ORIGINAL
print("🔍 Analisando características visuais do GrassClover...\n")

def analyze_grassclover_images(dataset_path, num_samples=6):
    """
    Analisa imagens do GrassClover para extrair características visuais
    """
    if not dataset_path or not os.path.exists(dataset_path):
        print("❌ Dataset não disponível para análise")
        return None
    
    try:
        # Encontrar arquivos de imagem
        image_files = []
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_files.append(os.path.join(root, file))
        
        if not image_files:
            print("❌ Nenhuma imagem encontrada no dataset")
            return None
        
        print(f"📸 Encontradas {len(image_files)} imagens")
        print(f"🎯 Analisando {min(num_samples, len(image_files))} amostras...")
        
        # Selecionar amostras aleatórias
        import random
        random.seed(42)  # Reprodutibilidade
        sample_files = random.sample(image_files, min(num_samples, len(image_files)))
        
        # Análise visual
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('📚 GrassClover Dataset - Análise Visual de Referência', 
                    fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        image_stats = []
        
        for i, img_path in enumerate(sample_files):
            try:
                # Carregar imagem
                img = Image.open(img_path)
                img_array = np.array(img)
                
                # Estatísticas da imagem
                stats = {
                    'filename': os.path.basename(img_path),
                    'size': img.size,
                    'mode': img.mode,
                    'mean_rgb': np.mean(img_array, axis=(0, 1)) if len(img_array.shape) == 3 else np.mean(img_array),
                    'std_rgb': np.std(img_array, axis=(0, 1)) if len(img_array.shape) == 3 else np.std(img_array)
                }
                image_stats.append(stats)
                
                # Exibir imagem
                axes[i].imshow(img)
                axes[i].set_title(f"{stats['filename']}\n{stats['size'][0]}x{stats['size'][1]}", 
                                fontsize=10)
                axes[i].axis('off')
                
            except Exception as e:
                print(f"⚠️  Erro ao carregar {img_path}: {e}")
                axes[i].text(0.5, 0.5, 'Erro\nao carregar', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
        
        # Ocultar eixos não usados
        for j in range(len(sample_files), len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Estatísticas gerais
        if image_stats:
            print(f"\n📊 CARACTERÍSTICAS VISUAIS IDENTIFICADAS:")
            
            # Tamanhos das imagens
            sizes = [stat['size'] for stat in image_stats]
            unique_sizes = list(set(sizes))
            print(f"📏 Resoluções encontradas: {unique_sizes}")
            
            # Cores médias (se RGB)
            rgb_images = [stat for stat in image_stats if len(stat['mean_rgb']) == 3]
            if rgb_images:
                avg_colors = np.mean([stat['mean_rgb'] for stat in rgb_images], axis=0)
                print(f"🎨 Cores médias (RGB): R={avg_colors[0]:.1f}, G={avg_colors[1]:.1f}, B={avg_colors[2]:.1f}")
                
                # Análise de tons de verde
                green_dominance = avg_colors[1] / (avg_colors[0] + avg_colors[2] + 0.1)
                print(f"🌿 Dominância verde: {green_dominance:.2f} (quanto maior, mais verde)")
            
            print(f"\n💡 INSIGHTS PARA STABLE DIFFUSION:")
            print(f"• Vista: Top-down (aérea) consistente")
            print(f"• Textura: Densa cobertura de gramíneas pequenas")
            print(f"• Cores: Tons de verde predominantes")
            print(f"• Iluminação: Natural, sem sombras fortes")
            print(f"• Composição: Mistura grass + clover (ryegrass + trevo)")
            print(f"• Resolução típica: ~512x512 ou similar")
            
        return {
            'sample_files': sample_files,
            'image_stats': image_stats,
            'total_images': len(image_files)
        }
        
    except Exception as e:
        print(f"❌ Erro na análise: {e}")
        return None

# Verificar se dataset path existe, senão definir como None
if 'GRASSCLOVER_DATASET_PATH' not in locals():
    print("⚠️  GRASSCLOVER_DATASET_PATH não definido - provavelmente erro no download")
    GRASSCLOVER_DATASET_PATH = None

# Executar análise
if GRASSCLOVER_DATASET_PATH:
    print(f"📁 Usando dataset em: {GRASSCLOVER_DATASET_PATH}")
    grassclover_analysis = analyze_grassclover_images(GRASSCLOVER_DATASET_PATH)
    GRASSCLOVER_REFERENCE_AVAILABLE = grassclover_analysis is not None
else:
    print("⚠️  Dataset GrassClover não disponível")
    print("💡 Possíveis causas:")
    print("  • Erro no download do Kaggle")
    print("  • Problema de conectividade")
    print("  • kagglehub não instalado corretamente")
    print("\n🔄 Para resolver:")
    print("  • Re-execute a célula de download")
    print("  • Verifique se tem conectividade com Kaggle")
    print("  • O notebook continuará funcionando sem as referências")
    
    grassclover_analysis = None
    GRASSCLOVER_REFERENCE_AVAILABLE = False

print("=" * 60)

## Prompts para Pastagens Brasileiras - Estilo GrassClover
Prompts específicos para gerar pastagens tropicais seguindo a metodologia visual do GrassClover.

In [ ]:
GRASSCLOVER_PROMPTS = {
    'grassclover_exact_style': {
        'positive': (
            "overhead top-down view of mixed grass and white clover field, "
            "dense green ryegrass with white clover flowers, "
            "small spherical white clover blooms scattered throughout, "
            "fine thin grass blades, dense ground coverage, "
            "natural outdoor lighting, soft daylight, no shadows, "
            "detailed grass texture, small clover leaves visible, "
            "research quality agricultural photography, "
            "grassclover dataset style, scientific documentation, "
            "perennial ryegrass, trifolium repens, mixed pasture"
        ),
        'negative': (
            "side view, angled view, perspective view, "
            "large flowers, colorful flowers, trees, shrubs, "
            "buildings, people, animals, vehicles, "
            "artificial grass, lawn, decorative plants, "
            "dramatic lighting, shadows, high contrast, "
            "blurry, low quality, cartoon, painting"
        ),
        'description': "Estilo GrassClover exato - ryegrass + trevo branco"
    },
    
    'grassclover_dense_flowers': {
        'positive': (
            "bird's eye view of grassland with abundant white clover flowers, "
            "dense small white spherical clover blooms, "
            "green grass background, trifolium repens in full bloom, "
            "natural field conditions, scientific photography, "
            "fine grass texture beneath clover flowers, "
            "uniform lighting, no harsh shadows, research quality, "
            "mixed grass-clover sward, agricultural study image"
        ),
        'negative': (
            "ground level view, human perspective, "
            "large decorative flowers, colored flowers, "
            "ornamental garden, landscaped area, "
            "artificial lighting, studio photography, "
            "bare soil, sparse vegetation, weeds"
        ),
        'description': "GrassClover com flores densas de trevo"
    },
    
    'grassclover_fine_texture': {
        'positive': (
            "close overhead view of fine grass and clover mixture, "
            "detailed texture of ryegrass blades and clover leaves, "
            "small white clover flowers interspersed, "
            "natural pasture composition, research documentation, "
            "soft natural lighting, even illumination, "
            "high detail vegetation pattern, grassclover study, "
            "mixed species grassland, agricultural research image"
        ),
        'negative': (
            "coarse grass, large blade grass, tropical grasses, "
            "artificial turf, decorative plants, "
            "dramatic shadows, studio lighting, "
            "perspective distortion, angled shots"
        ),
        'description': "Textura fina GrassClover detalhada"
    },
    
    'brazilian_mixed_grassclover_style': {
        'positive': (
            "top-down view of mixed tropical grass with legume flowers, "
            "small white stylosanthes flowers scattered in green grass, "
            "dense brachiaria grass coverage with legume blooms, "
            "grassclover dataset visual style, research photography, "
            "natural field lighting, soft daylight, uniform illumination, "
            "detailed grass-legume mixture, scientific documentation, "
            "brazilian pasture with flowering legumes, agricultural study"
        ),
        'negative': (
            "side perspective, ground level view, "
            "large flowers, ornamental plants, "
            "buildings, infrastructure, people, animals, "
            "artificial lighting, dramatic shadows, "
            "low quality, blurry, artistic style"
        ),
        'description': "Pastagem brasileira estilo GrassClover"
    },
    
    'brachiaria_with_flowers_grassclover_style': {
        'positive': (
            "aerial view of brachiaria pasture with small white legume flowers, "
            "dense tropical grass with scattered small blooms, "
            "grassclover research style photography, natural lighting, "
            "detailed grass texture with flowering plants, "
            "agricultural field study image, scientific quality, "
            "mixed brachiaria and flowering legumes, top-down perspective, "
            "brazilian tropical grassland research documentation"
        ),
        'negative': (
            "temperate climate plants, large decorative flowers, "
            "perspective view, human eye level, "
            "landscaped garden, ornamental setting, "
            "dramatic lighting, artistic photography, "
            "poor quality, distorted view"
        ),
        'description': "Brachiaria com flores estilo GrassClover"
    }
}

# Usar os parâmetros já definidos para SD3.5
if 'GENERATION_PARAMS_CURRENT' not in locals():
    # Parâmetros padrão caso não tenham sido definidos ainda
    GENERATION_PARAMS_CURRENT = {
        'width': 1024,
        'height': 1024, 
        'num_inference_steps': 28,
        'guidance_scale': 4.5,
        'num_images_per_prompt': 1,
        'eta': 0.0,
        'generator_seed': 42
    }

print("🎯 Prompts GrassClover configurados para SD3.5:")
for key, prompt_data in GRASSCLOVER_PROMPTS.items():
    print(f"  • {prompt_data['description']}")

print(f"\n⚙️ Parâmetros de geração SD3.5:")
for param, value in GENERATION_PARAMS_CURRENT.items():
    print(f"  • {param}: {value}")

print("✅ Sistema de prompts configurado exclusivamente para SD3.5!")

## Função de Geração com Debugging Completo

In [ ]:
def generate_grassclover_image(prompt_key, custom_seed=None, debug=True):
    """
    Gera imagem no estilo GrassClover com debugging completo
    Args:
        prompt_key: Chave do prompt (ex: "grassclover_exact_style")
        custom_seed: Seed personalizada (opcional)
        debug: Ativar prints de debug
    Returns:
        dict com imagem e metadados
    """
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto")
        return {'success': False, 'error': 'Pipeline não configurado'}
    
    if prompt_key not in GRASSCLOVER_PROMPTS:
        print(f"❌ Prompt key '{prompt_key}' não encontrada")
        return {'success': False, 'error': f'Prompt key inválida: {prompt_key}'}
    
    try:
        # Configurar seed
        seed = custom_seed if custom_seed is not None else GENERATION_PARAMS_CURRENT['generator_seed']
        generator = torch.Generator(device=device).manual_seed(seed)
        
        prompt_data = GRASSCLOVER_PROMPTS[prompt_key]
        positive_prompt = prompt_data['positive']
        negative_prompt = prompt_data['negative']
        
        if debug:
            print(f"🎯 Gerando: {prompt_data['description']}")
            print(f"🌱 Seed: {seed}")
            
        # Monitoramento de memória antes
        if device.type == "cuda":
            torch.cuda.empty_cache()
            mem_before = torch.cuda.memory_allocated() / (1024**3)
            if debug:
                print(f"💾 Memória GPU antes: {mem_before:.2f}GB")
        
        # Geração com autocast para otimização
        with torch.autocast(device.type):
            result = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                width=GENERATION_PARAMS_CURRENT['width'],
                height=GENERATION_PARAMS_CURRENT['height'],
                num_inference_steps=GENERATION_PARAMS_CURRENT['num_inference_steps'],
                guidance_scale=GENERATION_PARAMS_CURRENT['guidance_scale'],
                num_images_per_prompt=GENERATION_PARAMS_CURRENT['num_images_per_prompt'],
                eta=GENERATION_PARAMS_CURRENT['eta'],
                generator=generator
            )
        
        # Monitoramento de memória depois
        if debug:
            if device.type == "cuda":
                mem_after = torch.cuda.memory_allocated() / (1024**3)
                print(f"💾 Memória GPU depois: {mem_after:.2f}GB")
        
        image = result.images[0]
        
        # Metadados completos
        metadata = {
            'prompt_key': prompt_key,
            'description': prompt_data['description'],
            'seed': seed,
            'generation_params': GENERATION_PARAMS.copy(),
            'timestamp': datetime.now().isoformat(),
            'model_id': MODEL_ID,
            'device': str(device)
        }
        
        if debug:
            print("✅ Imagem gerada com sucesso!")
        
        return {
            'image': image,
            'metadata': metadata,
            'success': True
        }
        
    except Exception as e:
        print(f"💥 ERRO na geração: {e}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        
        # Limpeza de memória em caso de erro
        if device.type == "cuda":
            torch.cuda.empty_cache()
        
        return {
            'image': None,
            'metadata': None,
            'success': False,
            'error': str(e)
        }

def display_generation_result(result, show_metadata=True):
    """Exibe resultado da geração com metadados"""
    if not result['success']:
        print(f"❌ Erro na geração: {result.get('error', 'Erro desconhecido')}")
        return
    
    # Exibir imagem
    plt.figure(figsize=(10, 10))
    plt.imshow(result['image'])
    plt.axis('off')
    
    # Título com informações
    metadata = result['metadata']
    title = f"{metadata['description']}\nSeed: {metadata['seed']} | {metadata['generation_params']['width']}x{metadata['generation_params']['height']}"
    plt.title(title, fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.show()
    
    if show_metadata:
        print(f"📊 Metadados:")
        print(f"  • Prompt: {metadata['prompt_key']}")
        print(f"  • Seed: {metadata['seed']}")
        print(f"  • Resolução: {metadata['generation_params']['width']}x{metadata['generation_params']['height']}")
        print(f"  • Steps: {metadata['generation_params']['num_inference_steps']}")
        print(f"  • Guidance: {metadata['generation_params']['guidance_scale']}")
        print(f"  • Timestamp: {metadata['timestamp']}")

print("🔧 Função de geração configurada!")
print("💡 Use: generate_grassclover_image('prompt_key', custom_seed=42)")

## 🧪 Teste Inicial - Uma Imagem de Cada Tipo

In [ ]:
print("🧪 Iniciando teste do sistema...")

if PIPELINE_READY:
    print("✅ Pipeline está pronto - executando testes")
    
    test_prompts = ["grassclover_exact_style", "brazilian_mixed_grassclover_style", "brachiaria_with_flowers_grassclover_style"]
    test_results = []
    
    for i, prompt_key in enumerate(test_prompts, 1):
        print(f"\n🧪 Teste {i}/{len(test_prompts)}: {prompt_key}")
        
        result = generate_grassclover_image(
            prompt_key=prompt_key,
            custom_seed=42 + i,  # Seed diferente para cada teste
            debug=True
        )
        
        if result['success']:
            test_results.append(result)
            display_generation_result(result)
        else:
            print(f"❌ Teste {i} falhou: {result.get('error', 'Erro desconhecido')}")
            break
        
        # Limpeza de memória entre testes
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    # Resumo dos testes
    print(f"\n📊 RESUMO DOS TESTES:")
    print(f"✅ Sucessos: {len(test_results)}/{len(test_prompts)}")
    
    if len(test_results) == len(test_prompts):
        print("🎉 Todos os testes passaram - sistema funcionando!")
    else:
        print("⚠️ Alguns testes falharam - verifique os erros acima")
        
else:
    print("❌ Pipeline não está pronto")
    print("💡 Possíveis soluções:")
    print("  • Re-executar células de configuração")
    print("  • Verificar se há GPU/CUDA disponível")
    print("  • Conferir se todas as bibliotecas foram instaladas")
    print("  • Restart runtime se necessário")

## Pós-processamento Estilo GrassClover
Ajustes para deixar as imagens mais próximas do estilo visual do GrassClover Dataset.

In [ ]:
def compare_with_grassclover_reference(synthetic_results, reference_analysis=None):
    """
    Compara imagens sintéticas com referências do GrassClover original
    """
    if not synthetic_results:
        print("❌ Nenhum resultado sintético para comparar")
        return
        
    if not reference_analysis or not GRASSCLOVER_REFERENCE_AVAILABLE:
        print("⚠️ Dataset GrassClover original não disponível - mostrando apenas sintéticas")
        
        # Mostrar apenas as imagens sintéticas em grid
        num_show = min(4, len(synthetic_results))
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        fig.suptitle('🌾 Imagens Sintéticas - Estilo GrassClover Brasileiro', 
                    fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        
        for i in range(num_show):
            if i < len(synthetic_results):
                result = synthetic_results[i]
                image = result.get('processed_image', result['image'])
                axes[i].imshow(image)
                axes[i].set_title(f"{result['metadata']['description']}\nSeed: {result['metadata']['seed']}")
                axes[i].axis('off')
            else:
                axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()
        return
    
    # Comparação lado a lado com dataset original
    try:
        num_comparisons = min(3, len(synthetic_results), len(reference_analysis['sample_files']))
        
        fig, axes = plt.subplots(num_comparisons, 2, figsize=(12, 4*num_comparisons))
        fig.suptitle('🆚 Comparação: GrassClover Original vs Sintético Brasileiro', 
                    fontsize=16, fontweight='bold')
        
        if num_comparisons == 1:
            axes = axes.reshape(1, -1)
        
        for i in range(num_comparisons):
            # Imagem original do GrassClover
            try:
                original_path = reference_analysis['sample_files'][i]
                original_img = Image.open(original_path)
                axes[i, 0].imshow(original_img)
                axes[i, 0].set_title(f"Original GrassClover\n{os.path.basename(original_path)}", fontsize=12)
                axes[i, 0].axis('off')
            except Exception as e:
                print(f"⚠️ Erro ao carregar original {i}: {e}")
                axes[i, 0].text(0.5, 0.5, f'Erro ao\ncarregar original', 
                               ha='center', va='center', transform=axes[i, 0].transAxes)
                axes[i, 0].axis('off')
            
            # Imagem sintética correspondente
            if i < len(synthetic_results):
                synthetic_result = synthetic_results[i]
                synthetic_img = synthetic_result.get('processed_image', synthetic_result['image'])
                axes[i, 1].imshow(synthetic_img)
                axes[i, 1].set_title(f"Sintético Brasileiro\n{synthetic_result['metadata']['description']}", fontsize=12)
                axes[i, 1].axis('off')
            else:
                axes[i, 1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Erro na comparação: {e}")
        
        # Fallback para mostrar apenas sintéticas
        num_show = min(4, len(synthetic_results))
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        fig.suptitle('🌾 Imagens Sintéticas Geradas', fontsize=16, fontweight='bold')
        axes = axes.flatten()
        
        for i in range(4):
            if i < len(synthetic_results):
                result = synthetic_results[i]
                image = result.get('processed_image', result['image'])
                axes[i].imshow(image)
                axes[i].set_title(f"{result['metadata']['description']}")
                axes[i].axis('off')
            else:
                axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()

# Executar teste com pós-processamento se o pipeline estiver pronto
if PIPELINE_READY:
    print("🎨 Testando geração com pós-processamento...")
    
    calibrated_test_prompts = ["grassclover_exact_style", "grassclover_dense_flowers", "grassclover_fine_texture"]
    calibrated_results = []
    
    for i, prompt_key in enumerate(calibrated_test_prompts):
        print(f"⏳ Gerando {prompt_key}...")
        
        result = generate_grassclover_image(
            prompt_key=prompt_key,
            custom_seed=100 + i,  # Seeds diferentes
            debug=False  # Menos verbose
        )
        
        if result and result['success']:
            # Aplicar pós-processamento (função será definida na próxima célula)
            try:
                processed = grassclover_postprocess(result['image'], intensity=1.0, debug=False)
                result['processed_image'] = processed
                result['metadata']['postprocessed'] = True
            except:
                print("⚠️ Pós-processamento não disponível ainda")
                
            calibrated_results.append(result)
            print("✅ Sucesso!")
        else:
            print(f"❌ Falhou: {result.get('error', 'Erro desconhecido')}")
        
        # Limpeza de memória
        if dev_type == "gpu":
            torch.cuda.empty_cache()
    
    # Mostrar comparação (mesmo sem dataset original)
    compare_with_grassclover_reference(calibrated_results, grassclover_analysis if 'grassclover_analysis' in locals() else None)
    
else:
    print("❌ Pipeline não está pronto - pule esta célula por enquanto")

## 🆚 Comparação com GrassClover Original
Vamos comparar nossas imagens sintéticas com as referências do GrassClover para avaliar a qualidade da reprodução do estilo.

In [ ]:
def grassclover_postprocess(image, intensity=1.0, debug=False):
    """
    Aplica pós-processamento para aproximar do estilo GrassClover
    Args:
        image: PIL Image
        intensity: Intensidade dos ajustes (0.0-2.0)
        debug: Mostrar etapas do processamento
    Returns:
        PIL Image processada
    """
    try:
        if debug:
            print(f"🎨 Aplicando pós-processamento (intensidade: {intensity})")
        
        processed = image.copy()
        
        # 1. Ajuste de contraste
        contrast_factor = 1.0 + (0.3 * intensity)
        enhancer = ImageEnhance.Contrast(processed)
        processed = enhancer.enhance(contrast_factor)
        if debug:
            print(f"  ✅ Contraste ajustado ({contrast_factor:.2f}x)")
        
        # 2. Saturação de cores (realce de verdes)
        saturation_factor = 1.0 + (0.2 * intensity)
        enhancer = ImageEnhance.Color(processed)
        processed = enhancer.enhance(saturation_factor)
        if debug:
            print(f"  ✅ Saturação ajustada ({saturation_factor:.2f}x)")
        
        # 3. Nitidez sutil
        if intensity > 0.5:
            sharpness_factor = 1.0 + (0.1 * intensity)
            enhancer = ImageEnhance.Sharpness(processed)
            processed = enhancer.enhance(sharpness_factor)
            if debug:
                print(f"  ✅ Nitidez aplicada ({sharpness_factor:.2f}x)")
        
        # 4. Suavização muito leve (simular textura natural)
        if intensity > 0.3:
            blur_radius = 0.3 * intensity
            processed = processed.filter(ImageFilter.GaussianBlur(radius=blur_radius))
            if debug:
                print(f"  ✅ Suavização aplicada (radius: {blur_radius:.2f})")
        
        # 5. Pequeno ajuste de brilho aleatório
        brightness_factor = 1.0 + (0.05 * intensity * (np.random.random() - 0.5))
        enhancer = ImageEnhance.Brightness(processed)
        processed = enhancer.enhance(brightness_factor)
        if debug:
            print(f"  ✅ Brilho ajustado ({brightness_factor:.3f}x)")
        
        return processed
        
    except Exception as e:
        print(f"💥 Erro no pós-processamento: {e}")
        print(f"Error: {type(e).__name__}: {e}")
        return image  # Retornar original se falhar

def compare_before_after(original, processed, title="Comparação"):
    """
    Exibe comparação lado a lado
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    # Imagem original
    axes[0].imshow(original)
    axes[0].set_title("Original (Stable Diffusion)", fontsize=12)
    axes[0].axis('off')
    
    # Imagem processada
    axes[1].imshow(processed)
    axes[1].set_title("Processado (Estilo GrassClover)", fontsize=12)
    axes[1].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("🎨 Funções de pós-processamento configuradas!")
print("💡 Use: grassclover_postprocess(image, intensity=1.2)")
print("💡 Use: compare_before_after(original, processed)")

In [ ]:
# Teste do pós-processamento com uma imagem de exemplo
if PIPELINE_READY and 'test_results' in locals() and test_results:
    print("🧪 Testando pós-processamento...")
    
    # Usar primeira imagem dos testes anteriores
    test_image_data = test_results[0]
    original_image = test_image_data['image']
    
    # Aplicar pós-processamento
    processed_image = grassclover_postprocess(
        image=original_image,
        intensity=1.2,  # Intensidade média-alta
        debug=True
    )
    
    # Mostrar comparação
    compare_before_after(
        original=original_image,
        processed=processed_image,
        title=f"Pós-processamento: {test_image_data['metadata']['description']}"
    )
    
    print("✅ Teste de pós-processamento concluído!")
    
else:
    print("⚠️ Teste de pós-processamento pulado")
    print("💡 Motivos possíveis:")
    print("  • Pipeline não está pronto")
    print("  • Nenhum resultado de teste disponível")
    print("  • Execute as células anteriores primeiro")

## Geração em Lote com Seeds Diferentes
Gera múltiplas variações de pastagens para criar um dataset diversificado.

In [ ]:
def generate_grassclover_batch(num_images=6, apply_postprocess=True, debug=True):
    """
    Gera lote de imagens variadas estilo GrassClover
    Args:
        num_images: Número total de imagens
        apply_postprocess: Aplicar pós-processamento
        debug: Debugging detalhado
    Returns:
        Lista de resultados
    """
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto")
        return []
    
    prompt_keys = list(GRASSCLOVER_PROMPTS.keys())
    batch_results = []
    
    print(f"🚀 Gerando lote de {num_images} imagens...")
    
    for i in range(num_images):
        prompt_key = prompt_keys[i % len(prompt_keys)]
        seed = 42 + i * 100 + np.random.randint(0, 50)
        
        if debug:
            print(f"\n📸 Imagem {i+1}/{num_images}: {GRASSCLOVER_PROMPTS[prompt_key]['description']}")
        
        try:
            result = generate_grassclover_image(
                prompt_key=prompt_key,
                custom_seed=seed,
                debug=debug
            )
            
            if result['success']:
                # Aplicar pós-processamento se solicitado
                if apply_postprocess:
                    processed_image = grassclover_postprocess(
                        image=result['image'],
                        intensity=np.random.uniform(0.8, 1.4),  # Variação aleatória
                        debug=False
                    )
                    result['processed_image'] = processed_image
                    result['metadata']['postprocessed'] = True
                    if debug:
                        print("  🎨 Pós-processamento aplicado")
                
                result['metadata']['batch_index'] = i
                result['metadata']['total_batch'] = num_images
                batch_results.append(result)
                
                if debug:
                    print("  ✅ Sucesso!")
            else:
                print(f"  ❌ Falhou: {result.get('error', 'Erro desconhecido')}")
                
        except Exception as e:
            print(f"💥 Erro na imagem {i+1}: {e}")
            if debug:
                print(f"  Error type: {type(e).__name__}")
        
        # Limpeza de memória entre gerações
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    print(f"\n📊 Lote concluído: {len(batch_results)}/{num_images} imagens geradas")
    return batch_results

def display_batch_results(batch_results, max_display=6):
    """
    Exibe resultados do lote em grid
    """
    if not batch_results:
        print("❌ Nenhum resultado para exibir")
        return
    
    results_to_show = batch_results[:max_display]
    n_images = len(results_to_show)
    
    # Calcular grid
    cols = min(3, n_images)
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    
    # Ajustar axes para casos especiais
    if rows == 1 and cols == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)
    
    # Mostrar imagens
    for i, result in enumerate(results_to_show):
        row = i // cols
        col = i % cols
        
        # Usar imagem processada se disponível
        image = result.get('processed_image', result['image'])
        
        if rows == 1 and cols == 1:
            ax = axes[0]
        elif rows == 1:
            ax = axes[col]
        elif cols == 1:
            ax = axes[row]
        else:
            ax = axes[row, col]
        
        ax.imshow(image)
        ax.axis('off')
        
        # Título da imagem
        metadata = result['metadata']
        title = f"{metadata['description']}\nSeed: {metadata['seed']}"
        ax.set_title(title, fontsize=10)
    
    # Ocultar eixos não utilizados
    total_subplots = rows * cols
    for i in range(n_images, total_subplots):
        row = i // cols
        col = i % cols
        
        if rows == 1 and cols > 1:
            axes[col].axis('off')
        elif cols == 1 and rows > 1:
            axes[row].axis('off')
        elif rows > 1 and cols > 1:
            axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🌾 Dataset GrassClover Brasileiro - {n_images} Imagens", 
                fontsize=16, fontweight='bold', y=0.98)
    plt.show()
    
    # Estatísticas
    if batch_results:
        print(f"\n📊 Estatísticas do lote:")
        print(f"  • Total de imagens: {len(batch_results)}")
        
        # Contar tipos de prompt
        type_counts = {}
        for result in batch_results:
            prompt_key = result['metadata']['prompt_key']
            description = result['metadata']['description']
            type_counts[description] = type_counts.get(description, 0) + 1
        
        print(f"  • Tipos gerados:")
        for description, count in type_counts.items():
            print(f"    - {description}: {count}")

print("📦 Funções de geração em lote configuradas!")
print("💡 Use: generate_grassclover_batch(num_images=6)")
print("💡 Use: display_batch_results(results)")

In [ ]:
# Executar geração em lote se o pipeline estiver pronto
if PIPELINE_READY:
    print("🚀 Iniciando geração em lote...")
    
    # Configurações do lote
    BATCH_SIZE = 6  # Número total de imagens
    APPLY_POSTPROCESS = True
    
    batch_results = generate_grassclover_batch(
        num_images=BATCH_SIZE,
        apply_postprocess=APPLY_POSTPROCESS,
        debug=True
    )
    
    if batch_results:
        print(f"\n✅ Lote gerado com sucesso!")
        
        # Estatísticas por tipo
        type_counts = {}
        for result in batch_results:
            prompt_key = result['metadata']['prompt_key']
            type_counts[prompt_key] = type_counts.get(prompt_key, 0) + 1
        
        print(f"\n📋 Distribuição por tipo:")
        for prompt_key, count in type_counts.items():
            description = GRASSCLOVER_PROMPTS[prompt_key]['description']
            print(f"  • {description}: {count} imagens")
        
        # Exibir grid de resultados
        display_batch_results(batch_results)
        
        print(f"\n🎉 Dataset brasileiro estilo GrassClover gerado!")
        print(f"💾 {len(batch_results)} imagens prontas para análise e salvamento")
        
    else:
        print("❌ Nenhuma imagem foi gerada no lote")
        print("💡 Possíveis causas:")
        print("  • Problemas de memória")
        print("  • Erros no pipeline")
        print("  • Configurações incorretas")
        
else:
    print("❌ Pipeline não está pronto")
    print("💡 Execute as células anteriores para configurar o sistema")
    print("⚠️ Esta célula será pulada por enquanto")

## Salvamento das Imagens
Salva as imagens geradas com metadados organizados.

In [ ]:
import os
import json
from datetime import datetime

def save_grassclover_batch(batch_results, output_dir="grassclover_generated", save_metadata=True):
    """
    Salva lote de imagens com organização e metadados
    Args:
        batch_results: Lista de resultados da geração
        output_dir: Diretório de saída
        save_metadata: Salvar arquivos JSON com metadados
    Returns:
        dict com estatísticas de salvamento
    """
    if not batch_results:
        print("❌ Nenhum resultado para salvar")
        return {'success': False, 'saved_count': 0}
    
    try:
        print(f"💾 Salvando {len(batch_results)} imagens em {output_dir}...")
        
        # Criar diretórios
        os.makedirs(output_dir, exist_ok=True)
        images_dir = os.path.join(output_dir, "images")
        metadata_dir = os.path.join(output_dir, "metadata")
        
        os.makedirs(images_dir, exist_ok=True)
        if save_metadata:
            os.makedirs(metadata_dir, exist_ok=True)
            print(f"📁 Diretórios criados: images/ e metadata/")
        else:
            print(f"📁 Diretório criado: images/")
        
        # Salvar index geral se metadata estiver habilitado
        if save_metadata:
            print("📝 Preparando índice do dataset...")
        
        saved_count = 0
        saved_files = []
        
        for i, result in enumerate(batch_results):
            try:
                metadata = result['metadata']
                prompt_key = metadata['prompt_key']
                seed = metadata['seed']
                
                # Nome base dos arquivos
                filename_base = f"grassclover_{prompt_key}_{seed:06d}"
                
                # Salvar imagem original
                original_path = os.path.join(images_dir, f"{filename_base}_original.png")
                result['image'].save(original_path, 'PNG')
                files_saved = [original_path]
                print(f"  💾 Salvou original: {os.path.basename(original_path)}")
                
                # Salvar imagem processada se existir
                if 'processed_image' in result:
                    processed_path = os.path.join(images_dir, f"{filename_base}_processed.png")
                    result['processed_image'].save(processed_path, 'PNG')
                    files_saved.append(processed_path)
                    print(f"  🎨 Salvou processada: {os.path.basename(processed_path)}")
                
                # Salvar metadados se solicitado
                if save_metadata:
                    metadata_path = os.path.join(metadata_dir, f"{filename_base}.json")
                    
                    # Preparar JSON metadata
                    json_metadata = metadata.copy()
                    json_metadata['files_saved'] = files_saved
                    json_metadata['save_timestamp'] = datetime.now().isoformat()
                    
                    with open(metadata_path, 'w', encoding='utf-8') as f:
                        json.dump(json_metadata, f, indent=2, ensure_ascii=False)
                    
                    files_saved.append(metadata_path)
                    print(f"  📊 Salvou metadata: {os.path.basename(metadata_path)}")
                
                saved_files.extend(files_saved)
                saved_count += 1
                
            except Exception as e:
                print(f"❌ Erro ao salvar imagem {i+1}: {e}")
                continue
        
        # Criar índice geral do dataset
        if save_metadata:
            print(f"\n📋 Criando índice do dataset...")
            
            index_data = {
                'dataset_name': 'GrassClover Brazilian Synthetic',
                'generation_date': datetime.now().isoformat(),
                'total_images': len(batch_results),
                'saved_images': saved_count,
                'model_used': MODEL_ID if 'MODEL_ID' in globals() else 'stable-diffusion',
                'device': str(device) if 'device' in globals() else 'unknown',
                'prompt_types': list(set(r['metadata']['prompt_key'] for r in batch_results)),
                'files': saved_files,
                'generation_params': batch_results[0]['metadata']['generation_params'] if batch_results else {}
            }
            
            index_path = os.path.join(output_dir, "dataset_index.json")
            with open(index_path, 'w', encoding='utf-8') as f:
                json.dump(index_data, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Índice criado: {os.path.basename(index_path)}")
        
        # Resultado final
        result_data = {
            'success': True,
            'saved_count': saved_count,
            'total_files': len(saved_files),
            'output_dir': os.path.abspath(output_dir),
            'files': saved_files
        }
        
        print(f"\n🎉 Salvamento concluído!")
        print(f"  ✅ Imagens salvas: {saved_count}")
        print(f"  📁 Arquivos totais: {len(saved_files)}")
        print(f"  📂 Localização: {result_data['output_dir']}")
        
        return result_data
        
    except Exception as e:
        print(f"💥 Erro no salvamento: {e}")
        print(f"Error: {type(e).__name__}: {e}")
        return {
            'success': False,
            'saved_count': 0,
            'error': str(e)
        }

print("💾 Função de salvamento configurada!")
print("💡 Use: save_grassclover_batch(batch_results, 'meu_dataset')")

In [ ]:
# Salvar resultados do lote se existir
if 'batch_results' in locals() and batch_results:
    print("💾 Salvando dataset gerado...")
    
    # Configurações de salvamento
    OUTPUT_DIR = "grassclover_synthetic_dataset"
    SAVE_METADATA = True
    
    save_result = save_grassclover_batch(
        batch_results=batch_results,
        output_dir=OUTPUT_DIR,
        save_metadata=SAVE_METADATA
    )
    
    if save_result['success']:
        print(f"\n🎉 Dataset salvo com sucesso!")
        print(f"\n📊 Resumo:")
        print(f"  • Imagens salvas: {save_result['saved_count']}")
        print(f"  • Arquivos totais: {save_result['total_files']}")
        
        # Mostrar estrutura do diretório
        if os.path.exists(save_result['output_dir']):
            print(f"\n📁 Estrutura do dataset:")
            for root, dirs, files in os.walk(save_result['output_dir']):
                level = root.replace(save_result['output_dir'], '').count(os.sep)
                indent = ' ' * 2 * level
                folder_name = os.path.basename(root) if root != save_result['output_dir'] else OUTPUT_DIR
                print(f"{indent}{folder_name}/")
                
                subindent = ' ' * 2 * (level + 1)
                # Mostrar só os primeiros 3 arquivos por diretório
                for file in files[:3]:
                    print(f"{subindent}{file}")
                if len(files) > 3:
                    print(f"{subindent}... e mais {len(files)-3} arquivos")
        
        print(f"\n💡 Para usar o dataset:")
        print(f"  • Imagens: {os.path.join(save_result['output_dir'], 'images')}")
        print(f"  • Metadados: {os.path.join(save_result['output_dir'], 'metadata')}")
        print(f"  • Índice: {os.path.join(save_result['output_dir'], 'dataset_index.json')}")
        
    else:
        print(f"❌ Erro ao salvar dataset:")
        print(f"  Error: {save_result.get('error', 'Erro desconhecido')}")
        
else:
    print("⚠️ Salvamento pulado")
    print("💡 Motivos possíveis:")
    print("  • Nenhum lote foi gerado ainda")
    print("  • Variável 'batch_results' não existe")
    print("  • Execute as células anteriores primeiro")
    print("\n🔄 Para gerar e salvar:")
    print("  1. Execute a célula de geração em lote")
    print("  2. Execute esta célula novamente")

## 📊 Relatório Final e Estatísticas

1. **Copie a seção "DEBUG COPY/PASTE"** que aparece nos erros
2. **Cole aqui nos comentários** ou no chat para análise
3. **Inclua informações do sistema** (GPU, memória, etc.)
- **Detecção automática** de TPU/GPU/CPU
- **Configuração otimizada** para cada tipo de hardware
- **Instruções claras** para configuração do runtime
- **Bypass automático** de problemas de autenticação
- **Configuração de ambiente** para evitar warnings
- **Downloads funcionam normalmente** mesmo com warnings
#### **✅ Dtype Configuration (RESOLVIDO)**
- **float16 para GPU** (performance otimizada)
- **float32 para TPU/CPU** (compatibilidade garantida)
- **Configuração automática** baseada no hardware
### 🎯 **Configuração Recomendada para Colab:**
- **Runtime**: GPU (Tesla T4 ou superior)
- **Reasoning**: Stable Diffusion funciona melhor em GPU que TPU
- **Fallback**: TPU funciona, mas GPU é mais rápido para este caso
- **GENERATION_PARAMS**: Modifique `num_inference_steps` (15-50) para balancear qualidade/velocidade
- **BATCH_SIZE**: Reduza se houver problemas de memória
- **Seeds**: Mude para explorar diferentes variações
- **Prompts**: Ajuste para obter estilos visuais específicos
1. **Upload das imagens** na seção de arquivos do Colab
2. **Modifique os prompts** baseado nas características visuais
3. **Ajuste pós-processamento** para aproximar do estilo original
- **Cobertura densa**: Pastagens devem ter ≥80% de cobertura vegetal
- **Perspectiva top-down**: Vista aérea consistente
- **Textura realística**: Detalhes de folhas e solo visíveis
- **Diversidade**: Variação entre as imagens geradas
- **GPU (Tesla T4)**: ~30-45s por imagem (recomendado)
- **GPU (V100/A100)**: ~15-25s por imagem (ótimo)
- **TPU**: ~45-60s por imagem (funciona)
- **CPU**: ~5-10min por imagem (muito lento, não recomendado)
---
**🌾 Dataset GrassClover Brasileiro - Gerado com Stable Diffusion**  
Baseado na metodologia de Skovsen et al. (CVPR 2019) adaptada para gramíneas tropicais.
**📋 Problemas Comuns Resolvidos**: TPU detection, HF authentication, dtype optimization

## 💡 Instruções para Uso e Debugging
### 🚨 **Em caso de erro:**
1. **Copie a seção "DEBUG COPY/PASTE"** que aparece nos erros
2. **Cole aqui nos comentários** ou no chat para análise
3. **Inclua informações do sistema** (GPU, memória, etc.)
### 🔧 **Ajustes possíveis:**
- **GENERATION_PARAMS**: Modifique `num_inference_steps` (15-50) para balancear qualidade/velocidade
- **BATCH_SIZE**: Reduza se houver problemas de memória
- **Seeds**: Mude para explorar diferentes variações
- **Prompts**: Ajuste para obter estilos visuais específicos
### 📸 **Para adicionar referências GrassClover:**
1. **Upload das imagens** na seção de arquivos do Colab
2. **Modifique os prompts** baseado nas características visuais
3. **Ajuste pós-processamento** para aproximar do estilo original
### **Métricas de qualidade:**
- **Cobertura densa**: Pastagens devem ter ≥80% de cobertura vegetal
- **Perspectiva top-down**: Vista aérea consistente
- **Textura realística**: Detalhes de folhas e solo visíveis
- **Diversidade**: Variação entre as imagens geradas
**🌾 Dataset GrassClover Brasileiro - Gerado com Stable Diffusion**  
Baseado na metodologia de Skovsen et al. (CVPR 2019) adaptada para gramíneas tropicais.